## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from NewConfig import g_key

In [2]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kupang,ID,-10.1667,123.5833,78.75,85,45,1.83,scattered clouds
1,1,Menongue,AO,-14.6585,17.6910,64.49,99,100,4.54,moderate rain
2,2,Ust-Nera,RU,64.5667,143.2000,4.12,94,100,8.14,overcast clouds
3,3,Shirak,AM,40.8404,43.9158,21.52,68,40,2.30,scattered clouds
4,4,Biloela,AU,-24.4167,150.5000,76.21,59,0,0.18,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))


What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <=max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kupang,ID,-10.1667,123.5833,78.75,85,45,1.83,scattered clouds
4,4,Biloela,AU,-24.4167,150.5000,76.21,59,0,0.18,clear sky
5,5,Tulagi,SB,-9.0996,160.1476,82.80,69,52,3.29,broken clouds
7,7,Paita,PE,-5.0892,-81.1144,76.14,73,38,18.01,scattered clouds
9,9,Maragogi,BR,-9.0122,-35.2225,77.92,86,99,8.68,light rain
13,13,Isangel,VU,-19.5500,169.2667,81.28,81,100,8.10,overcast clouds
17,17,Bayshore Gardens,US,27.4253,-82.5904,84.24,36,0,8.05,clear sky
23,23,Les Cayes,HT,18.2000,-73.7500,77.99,74,46,8.05,scattered clouds
24,24,Isangel,VU,-19.5500,169.2667,81.28,81,100,8.10,overcast clouds
32,32,Sibolga,ID,1.7427,98.7792,75.61,85,81,3.67,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                134
City                   134
Country                133
Lat                    134
Long                   134
Max Temp               134
Humidity               134
Cloudiness             134
Wind Speed             134
Current Description    134
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.isnull().sum().sum()

0

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Long"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
0,Kupang,ID,78.75,scattered clouds,-10.1667,123.5833,
4,Biloela,AU,76.21,clear sky,-24.4167,150.5000,
5,Tulagi,SB,82.80,broken clouds,-9.0996,160.1476,
7,Paita,PE,76.14,scattered clouds,-5.0892,-81.1144,
9,Maragogi,BR,77.92,light rain,-9.0122,-35.2225,
13,Isangel,VU,81.28,overcast clouds,-19.5500,169.2667,
17,Bayshore Gardens,US,84.24,clear sky,27.4253,-82.5904,
23,Les Cayes,HT,77.99,scattered clouds,18.2000,-73.7500,
24,Isangel,VU,81.28,overcast clouds,-19.5500,169.2667,
32,Sibolga,ID,75.61,broken clouds,1.7427,98.7792,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    long = row["Long"]
    params["location"] = f"{lat},{long}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the searc
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found...skipping")
            
hotel_df.head(10)
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
0,Kupang,ID,78.75,scattered clouds,-10.1667,123.5833,Hotel On The Rock by Prasanthi
4,Biloela,AU,76.21,clear sky,-24.4167,150.5000,Biloela Silo Motor Inn
5,Tulagi,SB,82.80,broken clouds,-9.0996,160.1476,Raiders Hotel & Dive
7,Paita,PE,76.14,scattered clouds,-5.0892,-81.1144,Hotel Náutico de Paita
9,Maragogi,BR,77.92,light rain,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
13,Isangel,VU,81.28,overcast clouds,-19.5500,169.2667,Tanna Lodge
17,Bayshore Gardens,US,84.24,clear sky,27.4253,-82.5904,Ramada by Wyndham Sarasota Waterfront
23,Les Cayes,HT,77.99,scattered clouds,18.2000,-73.7500,Villa Orphee
24,Isangel,VU,81.28,overcast clouds,-19.5500,169.2667,Tanna Lodge
32,Sibolga,ID,75.61,broken clouds,1.7427,98.7792,Hotel Syariah CN Darussalam


In [16]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum().sum()

0

In [19]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} ℉</dd>
<dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Long"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Long"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                               max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))